In [1]:
import numpy as np

In [2]:
from sklearn import datasets
iris = datasets.load_iris()
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [3]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [4]:
X = []
Y = []
for idx, item in enumerate(iris.target):
    if item != 0: #убираем setosa
        X.append(iris.data[idx])
        Y.append(item) 

X = np.asarray(X)
Y = np.asarray(Y)

In [5]:
#переклассифицируем в классы 0 и 1 вместо 2 и 3
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(Y)
le.classes_
Y = le.transform(Y)

### Градиентный спуск

In [97]:
def predict(theta, X):
    """
    Предсказываем класс с помощью сигмоиды
    """
    predict = theta[0] + theta[1] * X[:, 0] + theta[2] * X[:, 1] + theta[3] * X[:, 2] + theta[4] * X[:, 3]
    sigm = 1. / (1 + np.exp(-predict))
    return sigm

def get_min_grad(lr,iter_,treshhold):
    """
    Находим значение ф-ии потерь методом град спуска 
    """
    theta = []
    theta = np.random.normal(size=(5,))
    grad_loss_hist = []
    for _ in range(iter_):
        sigm = predict(theta, X)
        theta[0] -= lr * np.sum(sigm - Y)/len(sigm)
        theta[1] -= lr * np.sum((sigm - Y) * X[:, 0])/len(sigm)
        theta[2] -= lr * np.sum((sigm - Y) * X[:, 1])/len(sigm)
        theta[3] -= lr * np.sum((sigm - Y) * X[:, 2])/len(sigm)
        theta[4] -= lr * np.sum((sigm - Y) * X[:, 3])/len(sigm)
        loss = - np.mean(np.log(sigm) * Y + np.log(1 - sigm) * (1 - Y))
        grad_loss_hist.append(loss)
    if loss<treshhold:
        return [theta,grad_loss_hist]
    else:
        print(f'Минимум ф-ии не получен, превышено допустимое кол-во итераций')
        
def score_grad(theta,grad_loss_hist):
    """
    Считаем accuracy
    """
    err = 0
    for idx, item in enumerate(np.around(predict(theta, X))):
        if item != Y[idx]:
            err += 1
    return (len(grad_loss_hist)- err)/len(grad_loss_hist)

In [107]:
#Находим параметры модели, при которых достигли минимума
model = get_min_grad(0.05, 1000, 0.5)
model[0]

array([-1.11909049, -1.81556461, -1.54246855,  2.74533262,  2.11037568])

In [108]:
#Считаем наш score (accuracy) при полученных параметрах
score_grad(model[0],model[1])

0.997

### Nesterov momentum

In [102]:
def predict(theta, X):
    """
    Предсказываем класс с помощью сигмоиды
    """
    predict = theta[0] + theta[1] * X[:, 0] + theta[2] * X[:, 1] + theta[3] * X[:, 2] + theta[4] * X[:, 3]
    sigm = 1. / (1 + np.exp(-predict))
    return sigm

def get_min_nest(lr,iter_,treshhold,gamma):
    """
    Находим значение ф-ии потерь методом Nesterov momentum
    """
    theta = []
    theta = np.random.normal(size=(5,)) 
    nest_loss_hist = []
    vel_pred = np.zeros(5)
    vel = np.zeros(5)
    for _ in range(iter_):
        sigm = predict(theta, X)
        loss = - np.mean(np.log(sigm) * Y + np.log(1 - sigm) * (1 - Y))
        nest_loss_hist.append(loss)
        sigm = predict(theta - gamma * vel_pred, X)
        vel[0] = (gamma * vel_pred[0] + lr * np.sum(sigm - Y))/len(sigm)
        vel[1] = (gamma * vel_pred[1] + lr * np.sum((sigm - Y) * X[:, 0]))/len(sigm)
        vel[2] = (gamma * vel_pred[2] + lr * np.sum((sigm - Y) * X[:, 1]))/len(sigm)
        vel[3] = (gamma * vel_pred[3] + lr * np.sum((sigm - Y) * X[:, 2]))/len(sigm)
        vel[4] = (gamma * vel_pred[4] + lr * np.sum((sigm - Y) * X[:, 3]))/len(sigm)
        theta -= vel
        vel_pred = vel
    if loss<treshhold:
        return [theta,nest_loss_hist]
    else:
        print(f'Минимум ф-ии не получен, превышено допустимое кол-во итераций')
        
def score_nest(theta,nest_loss_hist):
    """
    Считаем accuracy
    """
    err = 0
    for idx, item in enumerate(np.around(predict(theta, X))):
        if item != Y[idx]:
            err += 1
    return (len(nest_loss_hist)- err)/len(nest_loss_hist)

In [103]:
#Находим параметры модели, при которых достигли минимума
model2 = get_min_nest(0.05,1000,0.5,0.975)
model2[0]

array([-2.26739732, -1.96555796, -0.92334972,  2.85445114,  1.96630439])

In [104]:
#Считаем наш score (accuracy) при полученных параметрах
score_nest(model2[0],model2[1])

0.997

### RMSProp

In [115]:
def predict(theta, X):
    """
    Предсказываем класс с помощью сигмоиды
    """
    predict = theta[0] + theta[1] * X[:, 0] + theta[2] * X[:, 1] + theta[3] * X[:, 2] + theta[4] * X[:, 3]
    sigm = 1. / (1 + np.exp(-predict))
    return sigm

def get_min_rmsp(lr,iter_,treshhold,gamma,eps):
    """
    Находим значение ф-ии потерь методом 
    """
    theta = []
    theta = np.random.normal(size=(5,)) 
    rmsp_loss_hist = []
    e_sq_grad = np.zeros(5)
    grad = np.zeros(5)
    for _ in range(iter_):
        sigm = predict(theta, X)
        loss = - np.mean(np.log(sigm) * Y + np.log(1 - sigm) * (1 - Y))
        rmsp_loss_hist.append(loss)
        grad[0] = np.sum(sigm - Y)/len(sigm)
        grad[1] = np.sum((sigm - Y) * X[:, 0])/len(sigm)
        grad[2] = np.sum((sigm - Y) * X[:, 1])/len(sigm)
        grad[3] = np.sum((sigm - Y) * X[:, 2])/len(sigm)
        grad[4] = np.sum((sigm - Y) * X[:, 3])/len(sigm)
        e_sq_grad = gamma * e_sq_grad + (1 - gamma)  * grad ** 2
        theta -= lr * grad / np.sqrt(e_sq_grad + eps)
    if loss<treshhold:
        return [theta,rmsp_loss_hist]
    else:
        print(f'Минимум ф-ии не получен, превышено допустимое кол-во итераций')
        
def score_rmsp(theta,rmsp_loss_hist):
    """
    Считаем accuracy
    """
    err = 0
    for idx, item in enumerate(np.around(predict(theta, X))):
        if item != Y[idx]:
            err += 1
    return (len(rmsp_loss_hist)- err)/len(rmsp_loss_hist)

In [116]:
#Находим параметры модели, при которых достигли минимума
model3 = get_min_rmsp(0.05,1000,0.5,0.975,0.00000001)
model3[0]

array([-9.286705  , -2.22363853, -4.01758041,  4.03265485,  8.72970204])

In [117]:
#Считаем наш score (accuracy) при полученных параметрах
score_rmsp(model3[0],model3[1])

0.998